Module importing

In [4]:
import sys
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import scipy.signal as signal
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset

sys.path.append('/workspace/my_DL_tool/model')
from autoencoder import Autoencoder
sys.path.append('/workspace/my_DL_tool/tool')
from ppg_signal_load import PPGDataset
from model_builder import k_fold_training
from performance import performance

Data prepairing

In [ ]:
#Import_data
def preprocessing1(item):
    N = 200
    item[0] = signal.resample(item[0],N)
    item[1] = signal.resample(item[1],N)
    item[0] = (item[0]-np.min(item[0]))/(np.max(item[0])-np.min(item[0]))
    item[1] = (item[1]-np.min(item[1]))/(np.max(item[1])-np.min(item[1]))
    if len(item[0]) == len(item[1]):
        return item

def preprocessing2(item):
    N = 1000
    if len(item[0]) != 1000:
        item[0] = item[0][0:1000]
        item[0] = (item[0]-np.min(item[0]))/(np.max(item[0])-np.min(item[0]))
    if len(item[1]) != 1000:
        item[1] = item[1][0:1000]    
        item[1] = (item[1]-np.min(item[1]))/(np.max(item[1])-np.min(item[1]))
    
    return item

dataset = PPGDataset('/workspace/my_DL_tool/data/data.json',preprocessing = preprocessing1)
index = random.randint(0, len(dataset)-1)
z = dataset[0][0].numpy().flatten()
print(z.shape)
plt.plot(dataset[0][0].numpy().flatten())
plt.plot(dataset[0][1].numpy().flatten())
plt.show()

N = len(dataset)
test_len = int(N*0.2)
train_len = N-test_len

train_data, test_data = random_split(dataset, [train_len, test_len])

Training

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
model = Autoencoder(layer_dims = [200,200,200,200,100,100,100,50,50])
optimizer = optim.Adam(model.parameters(), lr=0.0001)
builder = k_fold_training(train_data,1,criterion = nn.L1Loss())
builder.training(model,device,optimizer,num_epochs=500,batch=128,visualize=1)

Performance

In [ ]:
shower = performance()
shower.test(model,device,test_data,per_fnc = "series")
criterion = nn.L1Loss()
for i in range(100,200):
    y = test_data[i][1].numpy().flatten()
    y_hat = model.forward(test_data[i][0].to(device))
    loss = criterion(y_hat,test_data[i][1].to(device))
    if loss.cpu().detach().numpy() <0.15:
        print(loss.cpu().detach().numpy())
        plt.plot(test_data[i][0].to(device).cpu().detach().numpy().flatten())
        plt.plot(y)
        plt.plot(y_hat.cpu().detach().numpy().flatten())
        # plt.stem(model.hidden_layer.cpu().detach().numpy().flatten())
        plt.show()